In [1]:
# API接続設定のファイルを読み込む
import configparser

In [2]:
# レート取得のために必要なライブラリ
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import pandas as pd

In [5]:
# 設定情報を"config_v1.txt"から読み込む
config = configparser.ConfigParser()
config.read('config_v1.txt') # パスの指定が必要です
account_id = config['oanda']['account_id']
api_key = config['oanda']['api_key']
api = API(access_token=api_key)

In [12]:
# 取得したい時間範囲を指定
params = {
    "count": 5000, # データ数を指定
        "granularity": "M1" # 1分間隔
}

In [7]:
# APIへ過去データをリクエスト
r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
api.request(r)

{'instrument': 'USD_JPY',
 'granularity': 'M10',
 'candles': [{'complete': True,
   'volume': 17,
   'time': '2019-01-14T04:30:00.000000000Z',
   'mid': {'o': '108.136', 'h': '108.140', 'l': '108.130', 'c': '108.140'}},
  {'complete': True,
   'volume': 20,
   'time': '2019-01-14T04:40:00.000000000Z',
   'mid': {'o': '108.138', 'h': '108.144', 'l': '108.131', 'c': '108.136'}},
  {'complete': True,
   'volume': 44,
   'time': '2019-01-14T04:50:00.000000000Z',
   'mid': {'o': '108.138', 'h': '108.138', 'l': '108.116', 'c': '108.117'}},
  {'complete': True,
   'volume': 52,
   'time': '2019-01-14T05:00:00.000000000Z',
   'mid': {'o': '108.114', 'h': '108.116', 'l': '108.088', 'c': '108.114'}},
  {'complete': True,
   'volume': 32,
   'time': '2019-01-14T05:10:00.000000000Z',
   'mid': {'o': '108.111', 'h': '108.111', 'l': '108.086', 'c': '108.098'}},
  {'complete': True,
   'volume': 66,
   'time': '2019-01-14T05:20:00.000000000Z',
   'mid': {'o': '108.100', 'h': '108.128', 'l': '108.100'

In [13]:
# dataとしてリストへ過去レートを変換
data = []
for raw in r.response['candles']:
    data.append([raw['time'], raw['volume'], raw['mid']['o'], raw['mid']['h'], raw['mid']['l'], raw['mid']['c']])


In [21]:
# リストをPandasのDataFrame形式へ変換
df = pd.DataFrame(data)
df.columns = ['time', 'volume', 'open', 'high', 'low', 'close']
df = df.set_index('time')
df.head() #無くても可

,volume,open,high,low,close
time,,,,,
2019-01-14T04:30:00.000000000Z,17,108.136,108.140,108.130,108.140
2019-01-14T04:40:00.000000000Z,20,108.138,108.144,108.131,108.136
2019-01-14T04:50:00.000000000Z,44,108.138,108.138,108.116,108.117
2019-01-14T05:00:00.000000000Z,52,108.114,108.116,108.088,108.114
2019-01-14T05:10:00.000000000Z,32,108.111,108.111,108.086,108.098


In [22]:
# date方がストリングで扱いづらいので、綺麗にする
df.index = pd.to_datetime(df.index)
df.tail() #無くても可

,volume,open,high,low,close
time,,,,,
2019-03-01 21:10:00,50,111.960,111.964,111.946,111.955
2019-03-01 21:20:00,81,111.952,111.952,111.914,111.920
2019-03-01 21:30:00,38,111.923,111.950,111.923,111.946
2019-03-01 21:40:00,32,111.942,111.950,111.936,111.950
2019-03-01 21:50:00,83,111.948,111.948,111.903,111.903


In [23]:
# DataFrameからCSVへ書き出し (easy)
df.to_csv('api-usdjpy-M1-20190302.csv')